<a href="https://colab.research.google.com/github/rayasrujanareddy/ECC/blob/main/Lab_Tak_7_AP22110011502.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Write a program to perform a key exchange using ECC.

In [3]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

# Generate private keys for both parties
alice_private_key = ec.generate_private_key(ec.SECP256R1())
bob_private_key = ec.generate_private_key(ec.SECP256R1())

# Derive public keys
alice_public_key = alice_private_key.public_key()
bob_public_key = bob_private_key.public_key()

# Exchange public keys and compute shared secret
alice_shared_secret = alice_private_key.exchange(ec.ECDH(), bob_public_key)
bob_shared_secret = bob_private_key.exchange(ec.ECDH(), alice_public_key)

# Verify both computed the same shared secret
assert alice_shared_secret == bob_shared_secret

print("Shared secret established successfully:", alice_shared_secret.hex())


Shared secret established successfully: 65684ee5596506d103f3a8cf03ae456bb71ed0ac4dae17dd06bfee7e5de03b6a


2.Write a program to encrypt and decrypt the message “Hell0 SRM AP” using Elliptic Curve
Cryptography and AES.
Use two different curve and report the changes.

In [30]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

# Function to derive AES key from shared secret
def derive_key(shared_secret):
    return HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'AES key derivation'
    ).derive(shared_secret)

# Function to encrypt a message using AES-GCM
def encrypt_message(aes_key, message):
    iv = os.urandom(12)
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message.encode()) + encryptor.finalize()
    return iv, encryptor.tag, ciphertext

# Function to decrypt a message using AES-GCM
def decrypt_message(aes_key, iv, tag, ciphertext):
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(iv, tag))
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()

# Using two different curves
curves = [ec.SECP256R1(), ec.SECP384R1()]

for curve in curves:
    print(f"Using curve: {curve.name}")

    # Generate private keys for both parties
    alice_private_key = ec.generate_private_key(curve)
    bob_private_key = ec.generate_private_key(curve)

    # Derive public keys
    alice_public_key = alice_private_key.public_key()
    bob_public_key = bob_private_key.public_key()

    # Exchange public keys and compute shared secret
    alice_shared_secret = alice_private_key.exchange(ec.ECDH(), bob_public_key)
    bob_shared_secret = bob_private_key.exchange(ec.ECDH(), alice_public_key)

    # Verify both computed the same shared secret
    assert alice_shared_secret == bob_shared_secret

    # Derive AES key from shared secret
    aes_key = derive_key(alice_shared_secret)

    # Encrypt and decrypt the message
    message = "Hell0 SRM AP"
    iv, tag, ciphertext = encrypt_message(aes_key, message)
    decrypted_message = decrypt_message(aes_key, iv, tag, ciphertext)

    print("Original Message:", message)
    print("Ciphertext:", ciphertext.hex())
    print("Decrypted Message:", decrypted_message.decode())
    print("----------------------------------")


Using curve: secp256r1
Original Message: Hell0 SRM AP
Ciphertext: 60a97df5b38e95206f5c1c63
Decrypted Message: Hell0 SRM AP
----------------------------------
Using curve: secp384r1
Original Message: Hell0 SRM AP
Ciphertext: da04e010170bc3173000746c
Decrypted Message: Hell0 SRM AP
----------------------------------
